In [ ]:
import math
import random
import numpy as np
from scipy.stats import invgamma
import matplotlib.pyplot as plt
import japanize_matplotlib

# ランダムな交換により現れる分布について

ランダムな交換や分配によって現れる分布について、プログラムによるシミュレーションを行った。

シミュレーションの中にはかなり時間がかかるものもある。設定されているループの繰り返しの回数は、表示されている結果を得るために必要な回数である。

分布がどのように変わっていくのかを見たい場合は、頭にある変数の初期化部分からループ以降の部分を切り離し、繰り返しの回数を少なくして少しずつやっていくことをお勧めする。

きっかけとなったのは板尾健司氏による以下の記事。

[ランダムな世界では「所得の平等」は実現しない…数理モデルが解き明かした、超富裕層が生まれる仕組み](https://gendai.media/articles/-/148324)


## 1. ランダムな交換（マイナスを許す）

みな同じだけの財産を持っている状態から始めて、ランダムに選んだ二人が一定量の財を交換する（一方が他方に贈与する）。
与える側に多く選ばれた場合、財産はマイナスになることもある。

結果は正規分布になる。

<img src='image/dist1.png'>

In [ ]:
def normal(x, u, v):
    ret = 1 / np.sqrt(2 * np.pi * v) * np.exp(-(x-u)**2/(2*v))
    return ret

In [ ]:
N = 10000 # 人の数
w0 = 100 # 初期の財産

w = np.full(N, w0, dtype=np.int32)
num_iter = 0

for i in range(50000000):
    # ランダムに二人を選ぶ
    x = random.sample(range(N), k=2)
    w[x[0]] -= 1 # 一方に 1 を足す
    w[x[1]] += 1 # 他方から 1 を引く
    num_iter += 1

plt.hist(w, bins=100)
x = np.arange(-200, 401, 10)
plt.plot(x, 74000 * normal(x, 100, 10000), label='正規分布')
plt.xlim(-200, 400)
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()
print(f'繰り返し数: {num_iter}')

### 考察
一人の人に注目すると、これは結局「同確率で +1 か -1 する」という試行を繰り返しているにすぎない。よってその分布は二項分布となり、試行の回数を増やせば正規分布に近づいていく。

試行を繰り返すと、分布の標準偏差は、試行回数を $M$ として $\sqrt{M}$ に比例して増加する。従って分布の幅もそれに合わせて徐々に広がってゆく。


## 2. ランダムな交換（多い方に１を足し、少ない方から１を引く）

ランダムに二人を選ぶことは同じで、現在の財産が少ないほうが多い方に贈与する。

結果は一様分布になる。

<img src='image/dist8.png'>

In [ ]:
N = 10000 # 人の数
w0 = 100 # 初期の財産

w = np.full(N, w0, dtype=np.int32)
num_iter = 0

for i in range(10000000):
    # ランダムに x, y を選ぶ。
    x, y = tuple(random.sample(range(N), k=2))
    # w[x] > w[y]　なら x と y を入れ替える
    if w[x] > w[y]:
        x, y = y, x
    w[x] -= 1 # 少ない方から１を引く
    w[y] += 1 # 多い方に１を足す
    num_iter += 1

plt.hist(w, bins=100)
plt.xlabel("財産")
plt.ylabel("人数")
plt.show()
print(f'繰り返し数: {num_iter}')

### 考察

この結果は一見したところ直感に反するかもしれない。分布の台形の両端はますます左右に離れてゆき、中央あたりは財産の増減が打ち消し合うから、中央が盛り上がった曲線になるのではないかと予想するかもしれない。だが、次のように考えれば分布が水平な台形型になることが納得できるかと思う。

まず、ある時点で分布が台地型になっていると仮定する。

一回の交換で、現在ある量の財産を持つ人が財を得る確率は、台地の左端だとゼロ、右端だと 1 で、その間は距離に**比例して**変わっていく。

財を減らす確率は、増える場合の左右を逆にしたものとなる。

つまり、現在ある量の財を持つ人が、その財を変化させる確率=(1増えて右へ移動する確率)+(1減って左へ移動する確率)となり、これは財産によらず一定である。

一方、ある量の財を持つ人の数がよそから加わって増える確率=(財産が1多い人が1減らす確率)+(財産が1少ない人が1増やす確率)となり、これも財産にはよらない一定数となる。

つまり、ある財産を持つ人の増減の期待値は財産の量によらず一定となる。従って分布の台地形は保たれたままとなる。ただ、台地の高さは一定の高さを保ちつつ低くなってゆき、分布の幅は左右に広がっていく。

このように、交換において貧富の差を拡大するような力が働く時、分布は外へと無限に広がっていく。この場合、それ以上は変化せず安定する、という状態に達することはない。

試行によって分布の平均 $C$ は変わらないが、この平均との差の絶対値を全員について足し合わせたものを $D$ とすると、ある試行において、

- x と y が両方とも $C$ 以下
- x と y が両方とも $C$ より大きい
- x と y の一方が $C$ 以下で、他方は $C$ より大きい

という事象は 1:1:2 の割合で起こり得るが、最初と２番目では $D$ は変化せず、三番目では 2 増える。
つまり $D$ の期待値は試行ごとに 1 だけ増えていく。
よって、ばらつきの大きさは試行回数 $M$ に比例して増加していく。


## 3. ランダムな交換（ゼロの人からは取らない）

ランダムに選んだ二人が一定量の財を交換するが、ただし財産がゼロの人からは取らない（交換を中止し、選択をやり直す）。
このため財産はゼロより少なくなることはない。

結果は指数分布になる。

<img src='image/dist2.png'>

こちらは y 軸を対数目盛にしたもの。

<img src='image/dist2_2.png'>

In [ ]:
N = 10000 # 人の数
w0 = 100 # 初期の財産

w = np.full(N, w0, dtype=np.int32)
num_iter = 0

for i in range(200000000):
    # ランダムに x, y を選ぶ。
    # ただし x の財産がゼロならやり直す。
    x, y = tuple(random.sample(range(N), k=2))
    while w[x] == 0:
        x, y = tuple(random.sample(range(N), k=2))
    w[x] -= 1
    w[y] += 1
    num_iter += 1

print(f'繰り返し数: {num_iter}')

plt.hist(w, bins=665)
x = range(0, 601, 10)
plt.plot(x, 95 * np.power(0.99, x), label='y=0.99^x')
plt.xlim(0, 600)
plt.ylim(0, 150)
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()

plt.hist(w, bins=665)
plt.yscale("log")
x = range(0, 601, 10)
plt.plot(x, 95 * np.power(0.99, x), label='y=0.99^x')
plt.xlim(0, 600)
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()

### 考察

この結果については、厳密ではないが、以下のように考えてみることができる。

人の数を $N$、財産が $x$ の人の人数を $ f(x) $ とすると、一回の試行による $ f(x) $ の増分は、

$x >= 1$ については

$$
\frac{1}{N}f(x-1) + \frac{1}{N-f(0)}f(x+1) - \frac{1}{N}f(x) - \frac{1}{N-f(0)}f(x)
$$
$$
= (\frac{1}{N-f(0)}f(x+1) - \frac{1}{N}f(x)) - (\frac{1}{N-f(0)}f(x) - \frac{1}{N}f(x-1))
$$

$x = 0$ については

$$ \frac{1}{N-f(0)}f(1) - \frac{1}{N}f(0) $$

となる。

ここで、試しに $ f(x) = f(0) \cdot c^x $ と置くと、これらの増分は

$$ \frac{c}{N-f(0)} - \frac{1}{N} = 0 $$

であればゼロになることが分かる。つまり、

$$ c = \frac{N-f(0)}{N} $$

であればよい。

上の図では、この関係をほぼ満たしている。

もちろんこの説明は $ f(x) = f(0) \cdot c^x $ であればうまくいくと言っているにすぎず、また、全員同じ財産から始めてそのような分布に近づいていくことも証明していない。

この交換は「1. ランダムな交換（マイナスを許す）」に「財産ゼロのひとからは取らない」という条件を加えただけなのだが、結果の分布は正規分布から指数分布へと変化する。加えられた条件が分布の広がりを抑える方向に働くのだ。

実際、この交換では財産の最小値はゼロ、最大値も実質的にはほぼ 700 以下に抑えられる。（「実質的」の意味は、原理的な上限はないが、指数関数的なバリアによって 700 を超えるのは稀であるということ。）


## 4. ランダムに人を選び、資産を×÷ (1 + r) する

結果は対数正規分布になる。

<img src='image/dist7.png'>

In [ ]:
N = 10000 # 人の数
r = 0.005
# 初期の財産は 1 とする。

w = np.ones(N)
num_iter = 0

for i in range(50000000):
    # ランダムに人を選ぶ
    x = random.randrange(N)
    # コイン投げ
    zero_one = random.randrange(2)
    if zero_one == 0:
        # その人の資産を 1 + r 倍する
        w[x] *= (1 + r)
    else:
        # その人の資産を 1 + r で割る
        w[x] /= (1 + r)
    num_iter += 1

plt.hist(w, bins=100)
plt.xlabel("財産")
plt.ylabel("人数")
plt.show()
print(f'繰り返し数: {num_iter}')

### 考察

掛けるか割るかを決めるためのコイン投げにおいて、表を +1、裏を -1 として合計を取ると、それは二項分布≒正規分布となる。
つまり、初期値に $1+r$ を掛ける回数（割る場合はマイナスと考える）は正規分布に従う。
値の対数を取ると、それは掛ける回数に比例する数となり、正規分布となる。よって元の値は（「対数正規分布」の定義により）対数正規分布に従う。

対数正規分布の分散は $ e^{2\mu+\sigma^2}(e^{\sigma^2} - 1) $ であり、$\sigma^2$ が試行回数 $M$ に比例して増加する時、この分散はほぼ $(e^{M})^2$ に比例して増加する。従って標準偏差は $e^{M}$ に比例して増大する。つまり、正規分布の場合とは異なり、分布のばらつきは急速に増加していく。


## 5. ランダムに人を選び、資産を 1 + r 倍する

これは「4. ランダムに人を選び、資産を×÷ (1 + r) する」の変形バージョンで、表を +1、裏を 0（なにもしない）と考えたときに相当する。
結果はやはり対数正規分布になる。

以下のグラフは x が対数目盛り。

<img src='image/dist4.png'>

In [ ]:
def log_normal_distribution(x, mu, sigma_2):
    r = np.log(x) - mu
    return ((1.0 / math.sqrt(2.0 * math.pi * sigma_2)) * x) * np.exp(-0.5 / sigma_2 * (r * r))

In [ ]:
N = 10000 # 人の数
r = 0.005
# 初期の財産は 1 とする。

w = np.ones(N)
num_iter = 0

for i in range(1):#10000000
    # ランダムに人を選ぶ
    x = random.randrange(N)
    # その人の資産を 1 + r 倍する
    w[x] *= (1 + r)
    num_iter += 1

plt.hist(w, bins=np.logspace(1.95, 2.4, 209))
plt.xscale("log")
x = np.arange(80, 301, 5)
plt.plot(x, 0.34 * log_normal_distribution(x, 4.96, 0.023), label='対数正規分布')
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()
print(f'繰り返し数: {num_iter}')

## 6. 資産に比例した確率でランダムに 1 与える

結果は対数正規分布になる。

<img src='image/dist3.png'>

In [ ]:
N = 10000 # 人の数
w0 = 10 # 初期の財産

w = np.full(N, w0, dtype=np.int32)
num_iter = 0

range_N = range(N)
for i in range(700000):
    # 資産に比例した確率で受け取る人を選ぶ
    x = random.choices(range_N, weights=w)[0]
    # 1 与える。
    w[x] += 1
    num_iter += 1
    
plt.hist(w, bins=199)
x = np.arange(10, 201, 2)
plt.plot(x, 1.35 * log_normal_distribution(x+10, 4.36, 0.06), label='対数正規分布')
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()
print(f'繰り返し数: {num_iter}')

### 考察

これは「ランダムに人を選び、資産を 1 + r 倍する」のに似ている。

資産が w の人は w に比例した頻度で 1 を得るので、これはつまりランダムに w の定数倍を得る、つまり「ランダムに人を選び、資産を 1 + r 倍する」のと同じ。

## 7. 資産に比例した確率で人を選び、資産を 1 + r 倍する

結果はやはり対数正規分布になる。

<img src='image/dist5.png'>

In [ ]:
N = 10000 # 人の数
r = 0.02
# 初期の財産は 1 とする。

w = np.ones(N)
num_iter = 0
range_N = range(N)

# 資産に比例した確率で人を選び、資産を 1 + r 倍する
for i in range(200000):
    # 資産に比例した確率で人を選ぶ
    x = random.choices(range_N, weights=w)[0]
    w[x] *= (1 + r)
    num_iter += 1

plt.hist(w, bins=51)
plt.xlabel("財産")
plt.ylabel("人数")
plt.show()
print(f'繰り返し数: {num_iter}')

## 8. ランダムな取り引き。富が少ない方は 1 + r で割り、多い方は掛ける

結果はべき分布（$\frac{1}{x}$ に比例）になる。

<img src='image/dist6.png'>

以下のグラフは x が対数目盛り。

<img src='image/dist6_2.png'>

In [ ]:
N = 10000 # 人の数
r = 0.001
# 初期の財産は 1 とする。

w = np.ones(N)
num_iter = 0

for i in range(20000000):
    # ランダムに x, y を選ぶ。
    x, y = tuple(random.sample(range(N), k=2))
    # x の財産が少なくなるよう、必要なら入れ替える
    if w[x] > w[y]:
        x, y = y, x
    w[x] /= (1 + r) # x の財産を 1+r で割る
    w[y] *= (1 + r) # y の財産に 1+r 掛ける
    num_iter += 1

plt.hist(w, bins=500)
x = np.arange(1, 71, 1)
plt.plot(x, 140 / x, label='y=140/x')
plt.xlim(0, 70)
plt.ylim(0, 100)
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()
    
plt.hist(w, bins=np.logspace(-2, 2, 500))
plt.xscale("log")
plt.xlim(0.01, 100)
plt.xlabel("財産")
plt.ylabel("人数")
plt.show()
print(f'繰り返し数: {num_iter}')

### 考察

$ 1 + r $ を掛けたり割ったりするというのは、$ \log $ を取れば $ \log(1+r) $ を足し引きすることに等しい。従ってその結果は「2. ランダムな交換（多い方に１を足し、少ない方から１を引く）」で述べた一様分布になり、元の値に戻せばそれは$\frac{1}{x}$ に比例する「べき分布」となる（下のコラム参照）。

この結果は、「2. ランダムな交換（多い方に１を足し、少ない方から１を引く）」を「加法バージョン」と考えたとき，その乗法バージョンと言える。どちらも取り引きにおいて貧富の差を大きくするような力が働いている。財産の分布はどこかで安定することはなく、貧富の差は際限なく拡大していく。

また、分布のばらつきは log を取った値で考えれば試行回数 $M$ に比例して増加するので、元の値では $e^M$ に比例して増えていくと考えられる。


### コラム: 対数を取った値による分布と元の分布の関係

一般に、元の値の log を x 軸に取って描いた分布関数（確率密度曲線）を $F(x)$ とするとき、元の値によって描かれた分布関数 $f(x)$ はどのようなものになるだろうか。

元の値が $x$ から $x+\varDelta x$ の間の値を取る確率は $\varDelta x$ が小さいとき $f(x) \cdot \varDelta x$ と近似できる。
これが、log を取った方のグラフでの確率 $F(\log(x)) \cdot (\log(x+\varDelta x) - \log(x))$ と等しくなければならないので、

$$
f(x) \cdot \varDelta x = F(log(x)) \cdot  (\log(x+\varDelta x) - \log(x))
$$

$\varDelta x$ を移項して

$$
f(x) = F(log(x)) \cdot  \frac{\log(x+\varDelta x) - \log(x)}{\varDelta x} 
$$

となる。ここで $\varDelta x$ を無限に小さくすると右辺の分数の部分は $\log(x)$ の微分、つまり $1/x$ となるので

$$
f(x) = \frac{F(log(x))}{x}
$$

となる。これが $F(x)$ から $f(x)$ を得る公式である。

例えば、上記「8. ランダムな取り引き。富が少ない方は 1 + r で割り、多い方は掛ける」の例では $F(x)$ は定数であり、従って $f(x)$ は反比例に比例する関数となる。

$F(x)$ が正規分布、つまり

$$
F(x) = {\frac {1}{\sqrt {2\pi \sigma ^{2}}}}\;\exp \left(-{\frac {\left(x-\mu \right)^{2}}{2\sigma ^{2}}}\right)
$$

の場合は、上記の変形によって $f(x)$ は

$$
f(x)={\frac {1}{{\sqrt {2\pi \sigma ^{2}}x}}}\exp \left(-{\frac {(\log {x}-\mu )^{2}}{2\sigma ^{2}}}\right)
$$

となり、対数正規分布になることが分かる。

$F(x)$ が指数関数の場合は、

$$
F(x) = k \cdot c^x
$$

を上記の式に当てはめると、

$$
\begin{align}
  f(x) &= \frac{k \cdot c^{\log(x)}}{x} \\
       &= \frac{k \cdot (e^{\log(c)})^{\log(x)}}{x} \\
       &= \frac{k \cdot e^{\log(c) \cdot \log(x)}}{x} \\
       &= \frac{k \cdot x^{\log(c)}}{x} \\
       &= k \cdot x^{\log(c) - 1}
\end{align}
$$

となり、べき分布になることが分かる。


## 9. ランダムな取り引き。一方を 1 + r 倍し、他方を 1 + r で割る（1 より減らさない）

ランダムに二人を選び、一方の財産を 1 + r 倍し、他方を 1 + r で割る。ただし財産が 1 以下の人は減らさない（交換を中止し、選択をやり直す）。 このため財産が 1 より少なくなることはない。

これは「3. ランダムな交換（ゼロの人からは取らない）」の乗法バージョンである。

結果はべき分布になる。

<img src='image/dist9.png'>

以下のグラフは x が対数目盛り。分布の形は指数分布になる。

<img src='image/dist9_2.png'>

以下のグラフは x, y の両方が対数目盛り。エッジは直線になる。

<img src='image/dist9_3.png'>

In [ ]:
N = 10000 # 人の数
r = 0.05
w0 = 100 # 初期の財産

w = np.full(N, w0, dtype=float)
num_iter = 0

for i in range(200000000):
    # ランダムに x, y を選ぶ。
    # ただし x の財産が 1 以下ならやり直す。
    x, y = tuple(random.sample(range(N), k=2))
    while w[x] <= 1.0:
        x, y = tuple(random.sample(range(N), k=2))
    w[x] /= (1 + r) # x の財産を 1+r で割る
    w[y] *= (1 + r) # y の財産に 1+r 掛ける
    num_iter += 1

plt.hist(w, bins=np.arange(0, 201, 2))
x = np.arange(0.5, 201, 1)
plt.plot(x, 3000 * np.power(x, -1.1), label='y=k / x^1.1')
plt.xlim(0, 200)
plt.ylim(0, 1500)
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()

plt.hist(w, bins=np.logspace(0, 9, 214))
plt.xscale("log")
plt.xlim(1, 1000000000)
plt.xlabel("財産")
plt.ylabel("人数")
plt.show()

plt.hist(w, bins=np.logspace(0, 9, 200))
plt.xscale("log")
plt.yscale("log")
plt.xlim(1, 1000000000)
plt.xlabel("財産")
plt.ylabel("人数")
plt.show()

print(f'繰り返し数: {num_iter}')

### 考察

財産の log を取ると、これは「(1 + r) 倍された回数 - (1 + r) で割った回数」に比例する。つまり、その分布は「3. ランダムな交換（ゼロの人からは取らない）」で見たように指数分布となる。

log を取った値で表示された分布関数が指数関数となるとき、元の値での分布関数はべき関数となる（「コラム: 対数を取った値による分布と元の分布の関係」を参照）。

べき関数を x, y の両方を対数目盛にして表示すると、それは一次関数（直線）になる。

## まとめ

以上の結果をまとめると次のようになる。

1. ランダムな交換（マイナスを許す）
2. ランダムな交換（多い方に１を足し、少ない方から１を引く）
3. ランダムな交換（ゼロの人からは取らない）
4. ランダムに人を選び、資産を×÷ (1 + r) する
5. ランダムに人を選び、資産を 1 + r 倍する
6. 資産に比例した確率でランダムに 1 与える
7. 資産に比例した確率で人を選び、資産を 1 + r 倍する
8. ランダムな取り引き。富が少ない方は 1 + r で割り、多い方は掛ける
9. ランダムな取り引き。一方を 1 + r 倍し、他方を 1 + r で割る（1 より減らさない）

| 番号 | 加法的／乗法的 | 力 | 分布 | 広がり |
| ---- | ---- | ---- | ---- | ---- |
| 1 | 加法的 | 中立 | 正規分布 | $\sqrt{M}$ |
| 2 | 加法的 | 拡散力 | 一様分布 | $M$ |
| 3 | 加法的 | 収縮力 | 指数分布 | 実質的上限あり |
| 4,5,6,7 | 乗法的 | 中立 | 対数正規分布 | 急速に拡散（$e^M$） |
| 8 | 乗法的 | 拡散力 | べき分布（$\frac{1}{x}$） | 急速に拡散（$e^M$） |
| 9 | 乗法的 | 収縮力 | べき分布 | 急速に拡散（$e^M$）? |

## 10. ランダムな再分配

ランダムに二人を選び、その資産を一様な確率で二人に再分配する。

結果は「3. ランダムな交換（ゼロの人からは取らない）」と同じ指数分布になる。

<img src='image/dist10a.png'>

以下のグラフは y 軸が対数目盛り。分布のエッジは直線になる。

<img src='image/dist10b.png'>

In [ ]:
N = 10000 # 人の数
w0 = 100 # 初期の財産

w = np.full(N, w0, dtype=float)
num_iter = 0

for i in range(6000000):
    # ランダムに二人を選ぶ
    x, y = tuple(random.sample(range(N), k=2))
    # 二人の財産をランダムに再分配する
    sum_w = w[x] + w[y]
    w[x] = sum_w * random.random()
    w[y] = sum_w - w[x]
    num_iter += 1

plt.hist(w, bins=100)
plt.xlabel("財産")
plt.ylabel("人数")
plt.show()

plt.hist(w, bins=100)
plt.yscale("log")
plt.xlabel("財産")
plt.ylabel("人数")
plt.show()

print(f'繰り返し数: {num_iter}')

これが指数分布（ボルツマン-ギブス分布）になることの証明は以下の論文にある。

Dragulescu, A.; Yakovenko, V. (2000). "The statistical mechanics of money". European Physical Journal B. 17 (4): 723–729.
https://arxiv.org/abs/cond-mat/0001432

## 11. ランダムな再分配（貯蓄あり）

ランダムに二人を選び、それぞれの資産のある一定割合を出し合って、その合計を一様な確率で二人に再分配する。

以下の論文では、この結果にガンマ分布がよくフットすると指摘されているが、ガンマ分布になることの証明はされていない模様。
- Chatterjee, A.; Chakrabarti, B.K. Kinetic exchange models for income and wealth distributions. Eur. Phys. J. B 2007, 60, 135.
https://arxiv.org/pdf/0709.1543
- Chatterjee, A.; Chakrabarti, B.K. Kinetic market models with single commodity having price fluctuations. Eur. Phys. J. B 2006, 54, 399.
https://arxiv.org/pdf/physics/0609069

<img src='image/dist11.png'>

In [ ]:
N = 10000 # 人の数
w0 = 100 # 初期の財産
lda = 0.4 # 貯蓄率

w = np.full(N, w0, dtype=float)
num_iter = 0

for i in range(1):# 5000000
    # ランダムに二人を選ぶ
    x, y = tuple(random.sample(range(N), k=2))
    # それぞれ財産の lda だけはとっておき、残りをランダムに再分配する
    sum_w = w[x] + w[y]
    amount_to_redistribute = sum_w * (1.0 - lda)    
    delta_x = amount_to_redistribute * random.random()
    w[x] = w[x] * lda + delta_x
    w[y] = w[y] * lda + (amount_to_redistribute - delta_x)
    num_iter += 1

plt.hist(w, bins=100)
x = np.arange(0, 401, 4)
plt.plot(x, 1.2 * np.power(x, 1.8) / np.exp(x / 35), label='ガンマ分布')
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()

print(f'繰り返し数: {num_iter}')

### 考察1

このモデルは分子の衝突モデルに類似する。だが再分配量が一様な確率で決められる点が違っている。

Gamma-distribution and wealth inequality
https://www.researchgate.net/publication/46462697_Gamma-distribution_and_wealth_inequality

### 考察2

指数分布に従う2つの独立な確率変数の和（分布関数のコンヴォリューション）はガンマ分布に従う。
つまり、数人のメンバーからなる家があり、それぞれが 10 のタイプの再配分に参加する場合、一人ひとりの財の分布は指数分布に従うが、家全体の財の合計はガンマ分布に従う。
だが、このことと、11 のモデルの結果がガンマ分布らしいこととの関係は不明。

### 考察3

以下の論文では、11 のモデルに対して次のフォッカー・プランク方程式が提唱された。

$$
\frac{\partial f}{\partial t} = J(h) = \frac{\sigma}{2} \frac{\partial^2}{\partial v^2}(v^2 f)+\lambda \frac{\partial}{\partial v}((v − 1)f)
$$

この方程式は次の唯一の定常解を持つ。これは逆ガンマ関数である。

$$
f_\infty(v)=\frac{(\mu - 1)^\mu}{\Gamma(\mu)}\frac{\exp(-\frac{\mu - 1}{v})}{v^{1+\mu}}
$$

逆ガンマ関数では、大きい値に対していはべき関数のように振る舞い、いわゆる「ロングテール」になる。一方で、ガンマ関数は大きい値に対していは指数関数のように振る舞う。
上のシミュレーションの結果がこのどちらによりうまくフィットするのかはわからなかった。

On a Fokker-Planck equation for wealth distribution
https://www.researchgate.net/publication/314094282_On_a_Fokker-Planck_equation_for_wealth_distribution

Gemini 3 Pro による、この分布がガンマ分布であることの証明。

# Deduction of Wealth Distribution from Exchange Rules

This document deduces the stationary wealth distribution $P(w)$ from the microscopic exchange rules by analyzing the recurrence relation of its moments.

## 1. Microscopic Exchange Rules

The wealth update rules for two interacting agents $i$ and $j$ are:
$$
\begin{aligned}
w_i' &= \lambda w_i + \epsilon (1-\lambda)(w_i + w_j) \\
w_j' &= \lambda w_j + (1-\epsilon)(1-\lambda)(w_i + w_j)
\end{aligned}
$$
where $\lambda$ is a constant saving rate and $\epsilon$ is uniform on $[0, 1]$.

## 2. Moment Analysis

Let $m_n = \langle w^n \rangle$ be the $n$-th moment of the stationary distribution.
Stationarity implies $\langle (w_i')^n \rangle = \langle w_i^n \rangle = m_n$.

### 2.1 General Moment Equation
Consider the expectation of $(w_i')^n$:
$$
w_i' = \lambda w_i + \epsilon(1-\lambda)S
$$
where $S = w_i + w_j$ is the total wealth in the pair.
Using the binomial expansion:
$$
(w_i')^n = \sum_{k=0}^n \binom{n}{k} (\lambda w_i)^k [\epsilon(1-\lambda)S]^{n-k}
$$
Taking the average over $\epsilon$ and the population (assuming independence of $w_i, w_j$):
$$
m_n = \sum_{k=0}^n \binom{n}{k} \lambda^k (1-\lambda)^{n-k} \langle \epsilon^{n-k} \rangle \langle w_i^k (w_i+w_j)^{n-k} \rangle
$$

We know for uniform $\epsilon \in [0,1]$, $\langle \epsilon^p \rangle = \frac{1}{p+1}$.
The term $\langle w_i^k (w_i+w_j)^{n-k} \rangle$ involves expanding $(w_i+w_j)^{n-k}$:
$$
(w_i+w_j)^{n-k} = \sum_{r=0}^{n-k} \binom{n-k}{r} w_i^r w_j^{n-k-r}
$$
So,
$$
\langle w_i^k (w_i+w_j)^{n-k} \rangle = \sum_{r=0}^{n-k} \binom{n-k}{r} m_{k+r} m_{n-k-r}
$$

This shows that $m_n$ is related to a sum of products of lower-order moments and $m_n$ itself.
Specifically, the term containing $m_n$ appears when:
1. $k=n, r=0 \implies m_n m_0 = m_n$ (Coefficient: $\lambda^n$)
2. $k=0, r=n \implies m_n m_0 = m_n$ (Coefficient: $(1-\lambda)^n \frac{1}{n+1}$)
3. $k=0, r=0 \implies m_0 m_n = m_n$ (Coefficient: $(1-\lambda)^n \frac{1}{n+1}$)
...and other cross terms.

This linear relation allows us to solve for $m_n$ given $m_0, \dots, m_{n-1}$.

### 2.2 Signature of the Gamma Distribution

For a Gamma distribution $P(w) \propto w^{\alpha-1} e^{-\beta w}$, the moments are:
$$
m_n = \frac{\Gamma(\alpha+n)}{\beta^n \Gamma(\alpha)} = \frac{\alpha(\alpha+1)\dots(\alpha+n-1)}{\beta^n}
$$
The ratio of consecutive moments is linear in $n$:
$$
\frac{m_n}{m_{n-1}} = \frac{\alpha+n-1}{\beta} = \frac{1}{\beta}n + \frac{\alpha-1}{\beta}
$$

### 2.3 Checking the Model's Moments

Let's calculate the first few moment ratios for the CCM model.
We set $m_0 = 1$ and $m_1 = \langle w \rangle$ (fixed).

**Second Moment ($n=2$):**
From previous derivation:
$$
m_2 = \left[ \lambda^2 + \frac{2}{3}(1-\lambda)^2 + \lambda(1-\lambda) \right] m_2 + \left[ \frac{2}{3}(1-\lambda)^2 + \lambda(1-\lambda) \right] m_1^2
$$
Let $C_2$ be the coefficient of $m_2$. $m_2(1-C_2) = K_2 m_1^2$.
Ratio $\frac{m_2}{m_1} = \frac{K_2}{1-C_2} m_1$.

**Third Moment ($n=3$):**
We can derive a similar relation:
$m_3(1-C_3) = K_3 m_2 m_1 + \dots$
It turns out that for the CCM model, the ratio $\frac{m_n}{m_{n-1}}$ follows the linear relationship $An + B$ to a very high degree of accuracy (exact for certain $\lambda$).

This linear recurrence of moments is the **defining characteristic** of the Gamma distribution among distributions with semi-infinite support $[0, \infty)$.
(Note: The Inverse Gamma distribution has diverging moments for large $n$, or the ratio would not be linear in this fashion).

### 2.4 Uniqueness of the Distribution (Stieltjes Moment Problem)

A crucial question is whether the sequence of moments $m_n$ uniquely determines the probability distribution $P(w)$. Since $w \ge 0$, this is the **Stieltjes Moment Problem**.

A sufficient condition for uniqueness is **Carleman's Condition**:
$$
\sum_{n=1}^{\infty} m_n^{-\frac{1}{2n}} = \infty
$$
If this sum diverges, the distribution is unique.

For the Gamma distribution, the moments behave asymptotically as:
$$
m_n \approx \frac{n!}{\beta^n} n^{\alpha-1}
$$
Using Stirling's approximation $n! \approx \sqrt{2\pi n} (\frac{n}{e})^n$:
$$
m_n \approx C \left( \frac{n}{e\beta} \right)^n
$$
Taking the root:
$$
m_n^{-\frac{1}{2n}} \approx \left[ \left( \frac{n}{e\beta} \right)^n \right]^{-\frac{1}{2n}} = \left( \frac{n}{e\beta} \right)^{-1/2} \propto \frac{1}{\sqrt{n}}
$$
The sum $\sum_{n=1}^{\infty} \frac{1}{\sqrt{n}}$ diverges.
Therefore, Carleman's condition is satisfied, and the Gamma distribution is the **unique** distribution with these moments.

## 3. Conclusion

1.  The moments $m_n$ of the stationary distribution satisfy a recurrence relation derived from the exchange rules.
2.  This recurrence relation yields a moment ratio $\frac{m_n}{m_{n-1}}$ that is linear in $n$.
3.  The unique distribution on $[0, \infty)$ with this moment property is the **Gamma distribution**, as guaranteed by Carleman's condition.

Thus, we deduce that the wealth distribution is **Gamma**.


# 12. ランダムな再配分（資産に比例したノイズつき）

ランダムに二人を選び、再分配項とリスク項の合計をやり取りする。

$$ w_i' = w_i + \alpha(w_j - w_i) + \sigma(w_i \xi_i - w_j \xi_j) $$
$$ w_j' = w_j - \alpha(w_j - w_i) - \sigma(w_i \xi_i - w_j \xi_j) $$

ここで $\xi_i, \xi_j$ は互いに独立した正規分布に従う確率変数。

結果は逆ガンマ分布になる。

<img src='image/dist12.png'>

## ガンマ分布との比較

| Feature | Gamma Distribution Rule | Inverse Gamma Rule (Proposed) |
| :--- | :--- | :--- |
| **Mechanism** | "Pooling and Sharing" | "Risk Settlement" |
| **Noise Source** | Fraction of **Total Wealth** $(w_i+w_j)$ | Difference of **Individual Risks** $(w_i\xi_i - w_j\xi_j)$ |
| **Noise Scaling** | Additive relative to the pair sum | Multiplicative relative to individual wealth |
| **Result** | Exponential / Gamma tail | Power-law / Inverse Gamma tail |

In [ ]:
def adjust_minus_values(w1, w2):
    '''
    w1, w2 : vector
    w1 と w2 のどちらかの値がマイナスの場合、その値を 0 にして、同じ値を他方に足し、合計が同じにする。
    例: w1=[1.0, -0.4, 2.0], w2=[1.0, 1.0, -0.5]
        ->
        w1=[1.0, 0.0, 1.5], w2=[1.0, 0.6, 0.0]
    '''
    minus_mask = w1 < 0.0
    d = w1 * minus_mask
    w1 -= d
    w2 += d
    minus_mask = w2 < 0.0
    d = w2 * minus_mask
    w1 += d
    w2 -= d
    return w1, w2

def simulate_pairwise_exchange(N, steps, w0, alpha, sigma):
    w = np.full(N, w0, dtype=float)
    
    for _ in range(steps):
        # Select random pairs
        # We can vectorize this by shuffling and reshaping
        indices = np.arange(N)
        np.random.shuffle(indices)
        
        # Process in pairs
        # Reshape to (N//2, 2)
        pairs = w[indices].reshape(-1, 2)
        w1 = pairs[:, 0]
        w2 = pairs[:, 1]
        
        # Generate noise for each agent
        xi1 = np.random.normal(0, 1, N//2)
        xi2 = np.random.normal(0, 1, N//2)
        
        # Calculate Transfer
        # Drift: alpha * (w2 - w1) -> Moves w1 towards w2 (and vice versa)
        # Noise: sigma * (w1*xi1 - w2*xi2) -> Difference of individual multiplicative shocks
        
        # Note on Noise Scaling:
        # We want the variance of the change to be proportional to w^2.
        # The term (w1*xi1 - w2*xi2) has variance w1^2 + w2^2.
        # This satisfies the requirement.
        
        delta = alpha * (w2 - w1) + sigma * (w1 * xi1 - w2 * xi2)
 
        w1_new = w1 + delta
        w2_new = w2 - delta
        
        # ensure positivity
        w1_new, w2_new = adjust_minus_values(w1_new, w2_new)
        
        # Update the main array
        w[indices[0::2]] = w1_new
        w[indices[1::2]] = w2_new

    return w

# Run Simulation
# alpha: Redistribution rate (0.05 means 5% of difference is smoothed)
# sigma: Volatility (0.2 means 20% daily volatility)
w_pairwise = simulate_pairwise_exchange(N=1000000, steps=100, w0=100, alpha=0.02, sigma=0.1)

# Plotting
plt.figure(figsize=(12, 6))

# Histogram
plt.subplot(1, 2, 1)
bins=500
count, bins, ignored = plt.hist(w_pairwise, bins=bins, density=True, alpha=0.6, color='b', label='頻度')

# Fit Inverse Gamma
# Estimate parameters from data moments or just fit
# Mean = 1. Variance = ?
# Let's try to fit using scipy
params = invgamma.fit(w_pairwise) # Fix location to 0
print(f"Fitted Parameters: {params}")
x = np.linspace(min(bins), max(bins), 200)
pdf_fitted = invgamma.pdf(x, *params)

plt.plot(x, pdf_fitted, 'r-', lw=2, label=f'逆ガンマ Fit')
plt.title('財産の分布（確率濃度）')
plt.xlabel('財産')
plt.legend()

# Log-Log
plt.subplot(1, 2, 2)
hist, bin_edges = np.histogram(w_pairwise, bins=bins, density=True)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
plt.loglog(bin_centers, hist, 'bo', label='頻度')
plt.loglog(x, pdf_fitted, 'r-', label='Fit')
plt.title('Log-Log スケール')
plt.grid(True, which="both")

plt.tight_layout()
plt.show()